# Malaria Blood Cell Classification with ResNet-50 (TensorFlow/Keras)

This notebook trains and evaluates a ResNet-50 model to classify blood cell images as Parasitized vs Uninfected using a manifest-based `tf.data` pipeline (no image copying).

- Dataset path: `/Users/jitesh/Downloads/cell_images` (Parasitized/ and Uninfected/)
- Manifests: `data/manifests/train.csv`, `val.csv`, `test.csv`
- Image size: 224, Batch size: 32
- Two-phase training: head, then fine-tune last ResNet block
- Metrics: Accuracy, Precision, Recall, F1, ROC-AUC; Confusion Matrix saved to `reports/figures/confusion_matrix.png`


In [2]:
# leave these commented
# %pip install --upgrade pip
# %pip install -r ../requirements.txt

import os
from pathlib import Path
import sys
import numpy as np
import pandas as pd

# --- Add project root (parent of notebooks/) to Python path ---
ROOT = Path("..").resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
# --------------------------------------------------------------

# NumPy <-> TensorFlow compatibility shim
if not hasattr(np, "complex_"):
    np.complex_ = np.complex128

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

from src.datasets_tf import get_datasets_from_manifests
from src.model_tf import build_resnet50

print("TF:", tf.__version__)
print("Devices:", tf.config.list_physical_devices())
print("ROOT:", ROOT)

/Users/jitesh/Malaria-Blood-Cell-Classification-Using-ResNet-50/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


TF: 2.16.2
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
ROOT: /Users/jitesh/Malaria-Blood-Cell-Classification-Using-ResNet-50


## Paths and configuration

In [3]:
RAW_DIR = Path('/Users/jitesh/Downloads/cell_images')
MANIFEST_DIR = Path('../data/manifests')
MODELS_DIR = Path('../models')
REPORTS_DIR = Path('../reports')
FIG_DIR = REPORTS_DIR / 'figures'

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS_HEAD = 10
EPOCHS_FT = 10
LR_HEAD = 1e-4
LR_FT = 1e-5
PATIENCE = 4
SEED = 42

MODELS_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

MODEL_OUT = MODELS_DIR / 'best_resnet50.h5'
METRICS_TXT = REPORTS_DIR / 'metrics.txt'


## Create manifests (if missing)

In [4]:
# Generate manifests only if they do not exist yet.
if not (MANIFEST_DIR / 'train.csv').exists():
    MANIFEST_DIR.mkdir(parents=True, exist_ok=True)
    import subprocess, sys
    print('Creating manifests...')
    cmd = [sys.executable, '-m', 'src.create_manifests', '--raw_dir', str(RAW_DIR), '--out_dir', str(MANIFEST_DIR), '--val_size', '0.15', '--test_size', '0.15', '--seed', str(SEED)]
    print(' '.join(cmd))
    res = subprocess.run(cmd, capture_output=True, text=True)
    print(res.stdout)
    if res.returncode != 0:
        print(res.stderr)
        raise RuntimeError('Failed to create manifests')
else:
    print('Manifests already exist at', MANIFEST_DIR)


Manifests already exist at ../data/manifests


## Build datasets from manifests

In [5]:
train_ds, val_ds, test_ds, class_names = get_datasets_from_manifests(str(MANIFEST_DIR), img_size=IMG_SIZE, batch_size=BATCH_SIZE, seed=SEED)
class_names


2025-11-14 17:58:42.854862: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-11-14 17:58:42.855043: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-11-14 17:58:42.855046: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-11-14 17:58:42.855439: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-14 17:58:42.855449: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


['Parasitized', 'Uninfected']

## Build model and train (head phase)

In [ ]:
# Stage 1: train classification head only
model, base = build_resnet50(input_shape=(IMG_SIZE, IMG_SIZE, 3))
model.compile(optimizer=tf.keras.optimizers.Adam(LR_HEAD),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

callbacks = [
    EarlyStopping(monitor='val_auc', mode='max', patience=PATIENCE, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_auc', mode='max', factor=0.5, patience=max(1, PATIENCE-1), min_lr=1e-6),
    ModelCheckpoint(filepath=str(MODEL_OUT), monitor='val_auc', mode='max', save_best_only=True)
]

history_head = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_HEAD, callbacks=callbacks)
MODEL_OUT.exists(), MODEL_OUT

Epoch 1/10


2025-11-14 17:58:47.835996: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-11-14 17:59:00.575252: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:10: Filling up shuffle buffer (this may take a while): 553 of 1000
2025-11-14 17:59:01.725491: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


603/603 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step - accuracy: 0.7480 - auc: 0.8246 - loss: 0.6496

603/603 ━━━━━━━━━━━━━━━━━━━━ 267s 408ms/step - accuracy: 0.7481 - auc: 0.8247 - loss: 0.6493 - val_accuracy: 0.8882 - val_auc: 0.9713 - val_loss: 0.2679 - learning_rate: 1.0000e-04
Epoch 2/10


2025-11-14 18:03:24.612981: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:10: Filling up shuffle buffer (this may take a while): 410 of 1000
2025-11-14 18:03:35.144339: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:10: Filling up shuffle buffer (this may take a while): 548 of 1000
2025-11-14 18:03:38.135295: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


: 

## Fine-tune last ResNet block

In [ ]:
# Stage 2: extended fine-tuning of upper ResNet blocks

# Unfreeze all layers from conv4_block1_* onward (conv4 + conv5 blocks)
base.trainable = True
start_unfreeze = False
for layer in base.layers:
    name = layer.name
    if 'conv4_block1' in name:
        start_unfreeze = True
    layer.trainable = start_unfreeze

# Use a smaller learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(LR_FT),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

history_ft = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_FT, callbacks=callbacks)
model.save(MODEL_OUT)
MODEL_OUT.exists(), MODEL_OUT

## Evaluate on validation and test sets

In [ ]:
val_metrics = model.evaluate(val_ds, return_dict=True)
test_metrics = model.evaluate(test_ds, return_dict=True)
val_metrics, test_metrics


## Detailed metrics and confusion matrix (Test set)

In [ ]:
# Collect ground truth and predictions
y_true = []
y_prob = []
for batch, labels in test_ds:
    y_true.extend(labels.numpy().reshape(-1).astype(int).tolist())
    y_prob.extend(model.predict(batch, verbose=0).reshape(-1).tolist())

y_true = np.array(y_true)
y_prob = np.array(y_prob)
y_pred = (y_prob >= 0.5).astype(int)

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)
try:
    auc = roc_auc_score(y_true, y_prob)
except Exception:
    auc = float('nan')

print({'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1, 'auc': auc})

# Save metrics
REPORTS_DIR.mkdir(parents=True, exist_ok=True)
with open(METRICS_TXT, 'w') as f:
    f.write(f'Accuracy: {acc}
')
    f.write(f'Precision: {prec}
')
    f.write(f'Recall: {rec}
')
    f.write(f'F1: {f1}
')
    f.write(f'ROC-AUC: {auc}
')
print('Saved metrics to', METRICS_TXT)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(4,3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Parasitized','Uninfected'],
            yticklabels=['Parasitized','Uninfected'])
plt.ylabel('True')
plt.xlabel('Predicted')
plt.tight_layout()
fig_path = FIG_DIR / 'confusion_matrix.png'
plt.savefig(fig_path, dpi=150)
plt.show()
print('Saved confusion matrix to', fig_path)


## Training curves (loss and AUC)

Visualize how training and validation metrics evolve over epochs for the head-training and fine-tuning stages.

In [ ]:
# Plot training curves for head and fine-tuning phases

def _plot_history(ax, history, metric, label_prefix):
    if history is None:
        return
    values = history.history.get(metric)
    val_values = history.history.get('val_' + metric)
    if values is None:
        return
    epochs = range(1, len(values) + 1)
    ax.plot(epochs, values, label=f'{label_prefix} train')
    if val_values is not None:
        ax.plot(epochs, val_values, label=f'{label_prefix} val')

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Loss
_plot_history(axes[0], history_head, 'loss', 'head')
_plot_history(axes[0], history_ft, 'loss', 'ft')
axes[0].set_title('Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()

# AUC
_plot_history(axes[1], history_head, 'auc', 'head')
_plot_history(axes[1], history_ft, 'auc', 'ft')
axes[1].set_title('AUC')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('AUC')
axes[1].legend()

plt.tight_layout()
plt.show()

## Export model to TFLite

Convert the trained Keras model (`best_resnet50.h5`) into a TensorFlow Lite model for deployment.

In [ ]:
# Convert the trained Keras model to TFLite

keras_model_path = MODEL_OUT
assert keras_model_path.exists(), f"Model not found at {keras_model_path}"

tflite_model_path = MODELS_DIR / 'best_resnet50.tflite'

converter = tf.lite.TFLiteConverter.from_keras_model(tf.keras.models.load_model(keras_model_path))
# Enable float16 quantization for smaller size (optional)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print('Saved TFLite model to', tflite_model_path)

## Single-image inference utility

Use the trained Keras model (and optionally the TFLite model) to classify a single blood cell image by file path.

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess


def load_and_preprocess_image(path, img_size=IMG_SIZE):
    img_raw = tf.io.read_file(path)
    img = tf.io.decode_image(img_raw, channels=3, expand_animations=False)
    img = tf.image.resize(img, (img_size, img_size))
    img = tf.cast(img, tf.float32)
    img = resnet_preprocess(img[None, ...])  # add batch dim and preprocess
    return img


def predict_image_keras(model, path):
    img = load_and_preprocess_image(path)
    prob = float(model.predict(img, verbose=0)[0, 0])
    label = 'Parasitized' if prob >= 0.5 else 'Uninfected'
    return label, prob


def predict_image_tflite(tflite_path, path):
    interpreter = tf.lite.Interpreter(model_path=str(tflite_path))
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    img = load_and_preprocess_image(path)
    # Cast to expected dtype
    img = tf.cast(img, input_details[0]['dtype']).numpy()

    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    prob = float(output[0][0])
    label = 'Parasitized' if prob >= 0.5 else 'Uninfected'
    return label, prob


# Example usage (update `test_image_path` to a real image from your dataset)
test_image_path = str(RAW_DIR / 'Parasitized' / os.listdir(RAW_DIR / 'Parasitized')[0])
print('Test image:', test_image_path)

keras_label, keras_prob = predict_image_keras(model, test_image_path)
print('Keras model ->', keras_label, keras_prob)

if (MODELS_DIR / 'best_resnet50.tflite').exists():
    tflite_label, tflite_prob = predict_image_tflite(MODELS_DIR / 'best_resnet50.tflite', test_image_path)
    print('TFLite model ->', tflite_label, tflite_prob)

## Experiments summary and backbone comparison

Load `reports/experiments.csv` to compare different backbones and runs (ResNet-50 vs EfficientNetB0, etc.).

In [ ]:
import pandas as pd

experiments_path = Path('../reports/experiments.csv')
if not experiments_path.exists():
    print('No experiments.csv found at', experiments_path)
else:
    exp_df = pd.read_csv(experiments_path)
    display(exp_df.sort_values(['test_auc', 'val_auc'], ascending=False).reset_index(drop=True))

In [ ]:
# Simple backbone vs test AUC plot (if experiments are available)

if 'exp_df' in globals():
    plt.figure(figsize=(5, 4))
    summary = exp_df.groupby('backbone')['test_auc'].max().reset_index()
    plt.bar(summary['backbone'], summary['test_auc'])
    plt.ylabel('Best Test AUC')
    plt.xlabel('Backbone')
    plt.title('Backbone comparison (best run per model)')
    plt.ylim(0.0, 1.0)
    plt.show()

## Grad-CAM visualizations (model explainability)

In this section we generate Grad-CAM heatmaps for a few test images to see where the model is focusing when predicting Parasitized vs Uninfected.

In [ ]:
from src.gradcam import make_gradcam_heatmap, overlay_heatmap

# Utility to grab a few images and labels from the test dataset
sample_images = []
sample_labels = []
for batch_imgs, batch_labels in test_ds.take(1):
    sample_images = batch_imgs.numpy()
    sample_labels = batch_labels.numpy().reshape(-1).astype(int)

print("Sample batch shape:", sample_images.shape, sample_labels.shape)


In [ ]:
# Generate and plot Grad-CAM for a few samples

# Name of the last conv layer in ResNet50 base (Keras default)
LAST_CONV_LAYER_NAME = 'conv5_block3_out'

num_to_show = min(6, len(sample_images))
plt.figure(figsize=(12, 6))
for i in range(num_to_show):
    img = sample_images[i]
    label = sample_labels[i]

    # Our model expects preprocessed inputs; test_ds already passed through preprocess_input
    # so we directly feed `img` to Grad-CAM.
    heatmap = make_gradcam_heatmap(img[None, ...], model, LAST_CONV_LAYER_NAME, pred_index=0)

    # Convert to displayable RGB image (0-255)
    disp_img = np.clip((img + 1.0) * 127.5, 0, 255).astype("uint8") if img.max() <= 1.1 else img.astype("uint8")
    overlay = overlay_heatmap(heatmap, disp_img, alpha=0.4)

    plt.subplot(2, num_to_show // 2, i + 1)
    plt.imshow(overlay)
    plt.axis('off')
    plt.title('Label: ' + ('Parasitized' if label == 0 else 'Uninfected'))

plt.tight_layout()
plt.show()